# Table Of Content:
* Preliminaries
    * [Structure of the lab](#structure)
    * [Objective of the lab](#objective)
    * [How to work for the lab?](#howto)
    * [Setting up the scene](#scene)
* [Part 1: Geometry](#part1)
    * [I. Computing the target configurations (Inverse Geometry)](#IG)
    * [II. Motion planning](#motion_planning)
* [Part 2: Dynamics](#part2)
    * [I. From a path to a trajectory](#TO)
    * [II. Implementing a control law](#control)




# Structure of the lab:  <a class="anchor" id="structure"></a>
The lab is divided into separate python files, each designed for you to address a sub-problem atomically. These instructions will indicate where you should implement each task. At the end of each file, in the 'main' 
section you can locally test your functions. It is important that you **do not modify the names and signatures** of the methods provided: When marking the lab, I will in first instance run code that will use these functions to evaluate quantitatively the methods you proposed. 

# Objective of the lab <a class="anchor" id="objective"></a>
Use both effectors of the nextage robot to grab a box and bring it to a target location.
You will first plan a valid motion that brings the robot to a grasping configuration, then moves the box to 
a desired location while avoiding collisions. For this you will use a combination of motion planning and numerical optimisation.

Once this motion plan will be computed, you will test it in a dynamics simulator using a control method of your choice.
This will be the objective of part 2 of the lab.

In the optional part 3 of the lab (for exceptional marks), you will be asked to self-propose a more complex task to achieve, which will require you to implement additional features to your framework. 

# How to work for the lab? <a class="anchor" id="howto"></a>
You have been used to notebooks for the tutorials, and these instructions also take the form of a notebook. To implement this lab I personally chose to work directly with a python IDE and I recommend to do the same. I worked with spyder but any python IDE might work as well. You can decide to work using notebooks if you prefer, this is not a problem, as long as your final deliverables meet the requirement specification.

**In any case, keep your code in a versioning system**. You are free to use github gitlab or whatever service you are more comfortable working with. The easiest way to work is to "fork" the lab repository from github into your own account and adding this new repository as a remote server.



## Code production
You are free to reuse code from the tutorials or any other source as long as you explicitely **cite its origin both in the code and in your report**. You are free to use any method from the pinocchio API and to create as many methods as you would like. **If you want to use non-native python libraries**, we must discuss this. Remember that I will have to run your code to assess your lab!


## I don't like the approach you have proposed to solve the problem. Can I do my own thing?
Yes... and no. First of all, I would suggest that you discuss this with the TAs / myself before going for it. Secondly, you will see in the submission requirements that I only need some methods to be implemented for me to assess quantitatively your work. I consider that if all of these methods are implemented you followed the instructions. The report will then give you a chance to justify your approach. This should give you a lot of freedom. In particular at step 2 I propose to use motion planning to compute a reference path for the robot. If you choose a different approach it does not matter to me as long as we discussed it before and that it is not hard-coded somehow. 

# Setting up the scene <a class="anchor" id="scene"></a>

To ensure efficient use of resources and to prevent the unintentional creation of multiple Meshcat server instances, I recommend initializing the server through the command line. Here's how you can do it:

- Open a terminal. If you're on Ubuntu, you can quickly do this by pressing `ctrl + alt + t`.
- Enter the following command and press `Enter`:
   ```
   meshcat-server 
   ```
- Upon running the command, you'll receive an output that includes the "zmq_url". This is the address to which you will connect.

**Tip**: You can run terminal commands directly from the Jupyter Notebook by prefixing them with `!`. However, for the purpose of this lab, I recommend initializing the Meshcat server directly from the terminal.


In [1]:
!meshcat-server

Port: 6000 in use, trying another...
Port: 7000 in use, trying another...
zmq_url=tcp://127.0.0.1:6001
web_url=http://127.0.0.1:7001/static/
^C


A helper function has been implemented to automatically connect to meshcat server, load the scene and the robot, and setup the collision handler in pinocchio for you.

In [15]:
from tools import setupwithmeshcat
robot, cube, viz = setupwithmeshcat(url="tcp://127.0.0.1:6000")

Wrapper tries to connect to server <tcp://127.0.0.1:6000>
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


```setupwithmeshcat``` takes an optional parameter corresponding to the url (zmq_url) of the meshcat server (given as a string). If no url is provided, it uses the variable ```MESHCAT_URL``` defined in config.py . If for some reason the default url does not match the one you are using, you can either provide this url to ```setupwithmeshcat``` or simply replace the value of ```MESHCAT_URL```

In [16]:
from config import MESHCAT_URL
print(MESHCAT_URL)

tcp://127.0.0.1:6000


In [17]:
hasattr(viz.viewer, 'jupyter_cell') and viz.viewer.jupyter_cell()

## Description of the environment.
The environment is composed of a fixed table and an obstacle, as well as a cube, which you are supposed to bring 
to the green target.

## Description of the robot. 
The robot you will use for the lab is the Nextage robot from Kawada industries.
You can use both pinocchio and the URDF files (in TODO) to check the dimension of the robot configuration space.
The configuration velocity space has the same dimension as Nextage is only composed of revolute joints.

In [18]:
print(robot.model)

Nb joints = 16 (nq=15,nv=15)
  Joint 0 universe: parent=0
  Joint 1 CHEST_JOINT0: parent=0
  Joint 2 HEAD_JOINT0: parent=1
  Joint 3 HEAD_JOINT1: parent=2
  Joint 4 LARM_JOINT0: parent=1
  Joint 5 LARM_JOINT1: parent=4
  Joint 6 LARM_JOINT2: parent=5
  Joint 7 LARM_JOINT3: parent=6
  Joint 8 LARM_JOINT4: parent=7
  Joint 9 LARM_JOINT5: parent=8
  Joint 10 RARM_JOINT0: parent=1
  Joint 11 RARM_JOINT1: parent=10
  Joint 12 RARM_JOINT2: parent=11
  Joint 13 RARM_JOINT3: parent=12
  Joint 14 RARM_JOINT4: parent=13
  Joint 15 RARM_JOINT5: parent=14



You can also verify that in its default configuration the robot is in collision (with the table)

In [6]:
from tools import collision
collision(robot, robot.q0)

True

# Configuration and helper functions  <a class="anchor" id="config"></a>

I have modified the URDF files to add frames that are relevant for the tasks you need to accomplish.
On the robot, I have created fixed joints attached to the tip of each effectors, called ```LARM_EFF``` and ```RARM_EFF```. Note that because they are fixed joints, they do not appear in the robot model and do not change the dimension of the configuration space. These names are defined for you in the config.py file as well. 

In [7]:
from config import LEFT_HAND, RIGHT_HAND

print ("Left hand joint name: ", LEFT_HAND)


import pinocchio as pin
q = robot.q0.copy()

#update the frame positions in robot.data given q
pin.framesForwardKinematics(robot.model,robot.data,q)

#now let's print the placement attached to the right hand
print ("Left hand joint placement: ")
pin.computeJointJacobians(robot.model,robot.data,q)
frameid = robot.model.getFrameId(LEFT_HAND)
oMframe = robot.data.oMf[frameid] 
print(oMframe)



Left hand joint name:  LARM_EFF
Left hand joint placement: 
  R =
-3.67321e-06           -1            0
           1 -3.67321e-06            0
           0            0            1
  p = 0.452  0.28 0.851



Likewise the cube urdf (models/cubes/cube_small.urdf) also contains helpers joint that set a target location for the effectors. They are called ```LEFT_HOOK``` and ```RIGHT_HOOK``` in config.py.

I added meshcat helper functions that will allow you to display the associated frames if you wish.

In [8]:
from config import LEFT_HOOK, RIGHT_HOOK, CUBE_PLACEMENT, CUBE_PLACEMENT_TARGET

from tools import getcubeplacement, setcubeplacement
from setup_meshcat import updatevisuals

#We can access the current cube position using
oMcube  = getcubeplacement(cube) #origin of the cube
oMcubeL = getcubeplacement(cube, LEFT_HOOK) #placement of the left hand hook
oMcubeR = getcubeplacement(cube, RIGHT_HOOK) #placement of the right hand hook


#the cube position is updated using the following function:
setcubeplacement(robot, cube, CUBE_PLACEMENT)
#to update the frames for both the robot and the cube you can call
updatevisuals(viz, robot, cube, q)


# Part 1: Geometry <a class="anchor" id="part1"></a>
In this first part of the lab, we are concerned with the geometry of the robot and in motion planning in general.
We are going to compute a geometric path that will serve as a guide for the dynamic part. Concretely, your objective is to compute a collision free path for the robot such that it "grasps" the cube and carries it over to its target position. Again, no dynamic computations are required in this phase, we are only interested in configurations that should be collision free, respect joint limits and geometrically consistent in terms of placement; there is already some work to be done here!

## I. Computing the target configurations (Inverse Geometry) <a class="anchor" id="IG"></a>

Your first task is the following: write the functions that generate an initial and a goal configuration for the nextage robot, such that the ```LEFT_HAND``` and ```RIGHT_HAND```are aligned respectively with the ```LEFT_HOOK``` and ```RIGHT_HOOK``` frames when the cube is located at its starting position  (```CUBE_PLACEMENT```) and at its goal position (```CUBE_PLACEMENT_TARGET```)

For this implement the method ```computeqgrasppose``` in inverse_geometry.py 

The main method indicates how the method is going to be called to obtain the q0 and qe configurations:
```
q0 = computeqgrasppose(robot, q, cube, CUBE_PLACEMENT, viz)
qe = computeqgrasppose(robot, q, cube, CUBE_PLACEMENT_TARGET,  viz)
```

Of course, q0 and qe should be collision-free and respect the joint limits of the robot. Do not hard-code anything here. In my tests I will use different targets to test the generality of ```computeqgrasppose```.

q0 should look somehow like this:
<img src="./images/q0.png" alt="drawing" width="200"/>



**hints:** 
* If your configurations look unnatural, you probably want to somehow introduce a "postural bias" in whatever method you are using 
* From the obtained configurations, you can easily obtain the relative placement between both hands expressed in a specific frame. This might prove relevant later in the lab so you may want to store it somewhere (maybe in solution.py)


## II. Motion planning <a class="anchor" id="motion_planning"></a>

For section II, I am proposing one course of action to successfully achieve the planning of a motion such that the robot carries the cube from its initial configuration to the end configuration. You are free to choose another course of action, but I do not advise it unless you really know what you are doing. In any case talk to me before making that decision. No matter what approach you choose, the method ```computepath``` from path.py must be implemented as specified for you to get the points.

To compute a geometric path that represents a collision-free motion of our robot carrying the cube from q0 to qe, I suggest to use a sampling based motion planner. 

We will use the path.py file to write the motion planning algorithm

### II.a Sampling configurations
To generate configurations for the planner, I suggest a 2 step process: randomly sample configurations for the cube, then solve an inverse geometry problem to generate a valid pose on that location. Of course, check that the joint limits are respected and the configuration is collision-free before returning it.

**hint:** you are free to bound your problem by ensuring that the cube placements that you generate only occur at positions / orientations that you think are interesting.

### II.b Path projection
However to do this we need to enforce the constraint that every sampled configuration is such that the effectors are holding the cube.

Furthermore, the grasping constraint will apply to the complete path: a standard linear interpolation between two grasping configurations is not enough to guarantee that every interpolated configuration is such that the cube is grasped.
Write a function that, given two configurations q0 and q1 and a discretisation step, returns an interpolated path between q0 and q1 such that every configuration in the path corresponds to a grasping pose. If it is not possible to generate such path, it will return a flag indicating so and, depending on your own decision, either return the part of the path that is valid, or nothing

### II.c Solution path generation
With the methods produced in II.1 and II.2, you should now be able to implement a motion planner that generates a geometrically valid path between q0 and qe. You can probably reuse code from the motion planning tutorial here, or implement you own. 

**requirement:** For me to assess this part you are required to implement the method ```computepath``` according to its documentation

# Part 2: Dynamics <a class="anchor" id="part2"></a>
Now that you have computed a reference path for your robot, it is time to try controlling it in a dynamics simulator. We will use pybullet for this. In theory pybullet is already installed but if not you should install it with pip:

This second part consists in two tasks. First, converting your path into a time-param trajectory




In [19]:
!pip install pybullet

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


## I. From a path to a trajectory <a class="anchor" id="TO"></a>

## II. Implementing a control law <a class="anchor" id="control"></a>
From a path to a trajectory